In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.utils import get_file
from tensorflow.keras import backend as K
from PIL import Image, ImageDraw
import requests
from io import BytesIO

# 이미지 다운로드 및 전처리 함수
def download_and_preprocess_image(url):
    response = requests.get(url)

    try:
        response.raise_for_status()  # Check for HTTP errors
        img = Image.open(BytesIO(response.content))
        img = img.convert('RGB')  # Ensure the image is in RGB format
        img = img.resize((224, 224))
    except requests.exceptions.HTTPError as errh:
        print(f"HTTP Error: {errh}")
        return None
    except Exception as e:
        print(f"Error opening image: {e}")
        return None

    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)

    return img_array

# R-CNN 모델 구성
def build_rcnn_model():
    base_model = ResNet50(weights='imagenet')
    base_model.layers.pop()  # 마지막 레이어 제거

    for layer in base_model.layers:
        layer.trainable = False

    x = base_model.layers[-1].output
    x = Dense(21, activation='softmax')(x)  # 21은 클래스의 수, 실제 데이터에 맞게 조절

    model = Model(inputs=base_model.input, outputs=x)
    return model

# 객체 검출 함수
def detect_objects(img_array, model):
    preds = model.predict(img_array)
    class_idx = np.argmax(preds)
    class_output = model.output[:, class_idx]
    last_conv_layer = model.get_layer('conv5_block3_out')

    grads = K.gradients(class_output, last_conv_layer.output)[0]
    pooled_grads = K.mean(grads, axis=(0, 1, 2))
    iterate = K.function([model.input], [pooled_grads, last_conv_layer.output[0]])

    pooled_grads_value, conv_layer_output_value = iterate([img_array])

    for i in range(512):  # conv5_block3_out의 채널 수
        conv_layer_output_value[:, :, i] *= pooled_grads_value[i]

    heatmap = np.mean(conv_layer_output_value, axis=-1)
    heatmap = np.maximum(heatmap, 0)
    heatmap /= np.max(heatmap)

    return heatmap

# 시각화 함수
def visualize_heatmap(heatmap, img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)

    heatmap = np.uint8(255 * heatmap)
    heatmap = np.expand_dims(heatmap, axis=-1)
    heatmap = np.concatenate([heatmap, heatmap, heatmap], axis=-1)

    superimposed_img = np.uint8(heatmap * img_array)
    superimposed_img = Image.fromarray(superimposed_img)

    orig_img = Image.fromarray(np.uint8(img_array))
    orig_img.show()
    superimposed_img.show()

# 메인 함수
def main():
    # Replace this with the direct link to your image
    image_url = 'https://i.imgur.com/YOUR_IMAGE_ID.jpg'
    img_array = download_and_preprocess_image(image_url)

    if img_array is not None:
        model = build_rcnn_model()
        heatmap = detect_objects(img_array, model)
        visualize_heatmap(heatmap, 'path/to/your/image.jpg')  # 로컬 이미지의 경로로 변경

if __name__ == "__main__":
    main()

HTTP Error: 404 Client Error: Not Found for url: https://imgur.com/YOUR_IMAGE_ID
